In [1]:
#dependencias
import pandas as pd
import numpy as np
import re
import unicodedata

#carga de datos
path21 = '/home/ubuntu/projects/historical-analysis-management-tickets/data/processed/df2021.csv'
path22 = '/home/ubuntu/projects/historical-analysis-management-tickets/data/processed/df2022.csv'
path23 = '/home/ubuntu/projects/historical-analysis-management-tickets/data/processed/df2023.csv'
path24 = '/home/ubuntu/projects/historical-analysis-management-tickets/data/processed/df2024.csv'
path25 = '/home/ubuntu/projects/historical-analysis-management-tickets/data/processed/df2025.csv'


In [2]:
#dataframes de trabajo
df2021 = pd.read_csv(path21, index_col=0)
df2022 = pd.read_csv(path22, index_col=0)
df2023 = pd.read_csv(path23, index_col=0)
df2024 = pd.read_csv(path24, index_col=0)
df2025 = pd.read_csv(path25, index_col=0)

#diccionario de dataframes
dict_df = {
    2021: df2021,
    2022: df2022,
    2023: df2023,
    2024: df2024,
    2025: df2025
}

#estandarizacion de nombres 2.0
for year, df in dict_df.items():
    df.columns = [col.lower().replace(" ", "_") for col in df.columns]

In [4]:
#conversion de fechas y calculo de SLA
cols_fecha = ['ultima_modificacion', 'fecha_de_apertura', 'fecha_de_cierre', 'fecha_de_solucion']
cols_str = ['ubicacion', 'seguimientos__origen_de_la_solicitud']

for year, df in dict_df.items():
    print(f"año {year}:")
    for col in cols_fecha:
        print(f"convirtiendo columna {col}")
        df[col] = pd.to_datetime(df[col], dayfirst=True, errors='coerce')
    for col in cols_str:
        print(f"convirtiendo columna {col}")
        df[col] = df[col].astype(str)
    print("----"*10)

año 2021:
convirtiendo columna ultima_modificacion
convirtiendo columna fecha_de_apertura
convirtiendo columna fecha_de_cierre
convirtiendo columna fecha_de_solucion
convirtiendo columna ubicacion
convirtiendo columna seguimientos__origen_de_la_solicitud
----------------------------------------
año 2022:
convirtiendo columna ultima_modificacion
convirtiendo columna fecha_de_apertura
convirtiendo columna fecha_de_cierre
convirtiendo columna fecha_de_solucion
convirtiendo columna ubicacion
convirtiendo columna seguimientos__origen_de_la_solicitud
----------------------------------------
año 2023:
convirtiendo columna ultima_modificacion
convirtiendo columna fecha_de_apertura
convirtiendo columna fecha_de_cierre
convirtiendo columna fecha_de_solucion
convirtiendo columna ubicacion
convirtiendo columna seguimientos__origen_de_la_solicitud
----------------------------------------
año 2024:
convirtiendo columna ultima_modificacion
convirtiendo columna fecha_de_apertura
convirtiendo columna f

In [5]:
#arreglar texto con mal formato en estadisticas__hora_de_resolucion
def reparar_formato_tiempo(texto):
    if pd.isna(texto):
        return "0 minutos"
    
    texto = str(texto)
    reparaciones = {
        'Ã\xad': 'í',
        'daas': 'días',
        'dÃas': 'días',
        ' d ': ' días ',
    }
    
    for error, correccion in reparaciones.items():
        texto = texto.replace(error, correccion)
    
    texto = texto.lower().strip()
    
    if 'dias' in texto:
        texto = texto.replace('dias', 'días')
        
    return texto

# Aplicar la limpieza estética a la columna original
for year, df in dict_df.items():
    df['estadisticas__hora_de_resolucion'] = df['estadisticas__hora_de_resolucion'].apply(reparar_formato_tiempo)

In [6]:
#Transformacion columna de tiempo de resolucion str a fraccion de horas
def limpiar_y_convertir_a_horas(texto):
    if pd.isna(texto) or texto == '':
        return 0.0
    
    texto = str(texto)
    texto = unicodedata.normalize('NFKD', texto).encode('ascii', 'ignore').decode('utf-8')
    texto = texto.lower()
    
    dias = re.search(r'(\d+)\s*d', texto)
    horas = re.search(r'(\d+)\s*h', texto)
    minutos = re.search(r'(\d+)\s*m', texto)
    
    d_val = int(dias.group(1)) * 24 if dias else 0
    h_val = int(horas.group(1)) if horas else 0
    m_val = int(minutos.group(1)) / 60 if minutos else 0
    
    return round(d_val + h_val + m_val, 2)

# tranformacion de columna de horas a decimal
col_target = 'estadisticas__hora_de_resolucion'

for year, df in dict_df.items():
    df['resolucion_horas_decimal'] = df[col_target].apply(limpiar_y_convertir_a_horas)

In [7]:
#verificacion de modificaciones
for year, df in dict_df.items():
    print(f"Info del DataFrame {year}:")
    display(df.info())

Info del DataFrame 2021:
<class 'pandas.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 18 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   id                                    2 non-null      int64         
 1   titulo                                2 non-null      str           
 2   estado                                2 non-null      str           
 3   ultima_modificacion                   2 non-null      datetime64[us]
 4   fecha_de_apertura                     2 non-null      datetime64[us]
 5   prioridad                             2 non-null      str           
 6   solicitante__solicitante              2 non-null      str           
 7   asignado_a__grupo_de_tecnicos         2 non-null      str           
 8   asignado_a__tecnico                   2 non-null      str           
 9   categoria                             2 non-null      str         

None

Info del DataFrame 2022:
<class 'pandas.DataFrame'>
RangeIndex: 3827 entries, 0 to 3826
Data columns (total 18 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   id                                    3827 non-null   int64         
 1   titulo                                3827 non-null   str           
 2   estado                                3827 non-null   str           
 3   ultima_modificacion                   3827 non-null   datetime64[us]
 4   fecha_de_apertura                     3827 non-null   datetime64[us]
 5   prioridad                             3827 non-null   str           
 6   solicitante__solicitante              3772 non-null   str           
 7   asignado_a__grupo_de_tecnicos         3561 non-null   str           
 8   asignado_a__tecnico                   2580 non-null   str           
 9   categoria                             2956 non-null   str   

None

Info del DataFrame 2023:
<class 'pandas.DataFrame'>
RangeIndex: 36259 entries, 0 to 36258
Data columns (total 18 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   id                                    36259 non-null  int64         
 1   titulo                                36259 non-null  str           
 2   estado                                36259 non-null  str           
 3   ultima_modificacion                   36259 non-null  datetime64[us]
 4   fecha_de_apertura                     36259 non-null  datetime64[us]
 5   prioridad                             36259 non-null  str           
 6   solicitante__solicitante              35899 non-null  str           
 7   asignado_a__grupo_de_tecnicos         33944 non-null  str           
 8   asignado_a__tecnico                   30251 non-null  str           
 9   categoria                             34161 non-null  str 

None

Info del DataFrame 2024:
<class 'pandas.DataFrame'>
RangeIndex: 35004 entries, 0 to 35003
Data columns (total 18 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   id                                    35004 non-null  int64         
 1   titulo                                35004 non-null  str           
 2   estado                                35004 non-null  str           
 3   ultima_modificacion                   35004 non-null  datetime64[us]
 4   fecha_de_apertura                     35004 non-null  datetime64[us]
 5   prioridad                             35004 non-null  str           
 6   solicitante__solicitante              34858 non-null  str           
 7   asignado_a__grupo_de_tecnicos         33346 non-null  str           
 8   asignado_a__tecnico                   33356 non-null  str           
 9   categoria                             31389 non-null  str 

None

Info del DataFrame 2025:
<class 'pandas.DataFrame'>
RangeIndex: 36527 entries, 0 to 36526
Data columns (total 18 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   id                                    36527 non-null  int64         
 1   titulo                                36527 non-null  str           
 2   estado                                36527 non-null  str           
 3   ultima_modificacion                   36527 non-null  datetime64[us]
 4   fecha_de_apertura                     36527 non-null  datetime64[us]
 5   prioridad                             36527 non-null  str           
 6   solicitante__solicitante              36369 non-null  str           
 7   asignado_a__grupo_de_tecnicos         35292 non-null  str           
 8   asignado_a__tecnico                   35160 non-null  str           
 9   categoria                             34169 non-null  str 

None

In [8]:
#Calculo SLA (hipotetico)
OBJETIVO_SLA_HORAS = 24

for year, df in dict_df.items():
    df['cumple_SLA'] = df['resolucion_horas_decimal'] <= OBJETIVO_SLA_HORAS
    porcentaje_cumplimiento = df['cumple_SLA'].mean() * 100
    print(f"El cumplimiento general del SLA del {year}: {porcentaje_cumplimiento:.2f}%")
    print("----"*10)

El cumplimiento general del SLA del 2021: 0.00%
----------------------------------------
El cumplimiento general del SLA del 2022: 39.04%
----------------------------------------
El cumplimiento general del SLA del 2023: 76.76%
----------------------------------------
El cumplimiento general del SLA del 2024: 76.64%
----------------------------------------
El cumplimiento general del SLA del 2025: 81.36%
----------------------------------------


In [9]:
#tickets con mayor tiempo de resolucion prueba de parametro nuevo
col_target = 'estadisticas__hora_de_resolucion'

for year, df in dict_df.items():
    print(f"Año {year}:")
    df_lentos = df.nlargest(5, 'resolucion_horas_decimal')
    print("Tickets con mayor tiempo de resolución:")
    display(df_lentos[[col_target, 'resolucion_horas_decimal']])


Año 2021:
Tickets con mayor tiempo de resolución:


,estadisticas__hora_de_resolucion,resolucion_horas_decimal
0,575 días 15 horas 53 minutos,13815.88
1,559 días 6 horas 17 minutos,13422.28


Año 2022:
Tickets con mayor tiempo de resolución:


,estadisticas__hora_de_resolucion,resolucion_horas_decimal
110,604 días 17 horas 50 minutos,14513.83
111,604 días 17 horas 36 minutos,14513.60
112,604 días 16 horas 38 minutos,14512.63
134,225 días 5 horas 9 minutos,5405.15
133,225 días 0 horas 32 minutos,5400.53


Año 2023:
Tickets con mayor tiempo de resolución:


,estadisticas__hora_de_resolucion,resolucion_horas_decimal
926,597 días 19 horas 57 minutos,14347.95
929,597 días 17 horas 49 minutos,14345.82
930,596 días 22 horas 36 minutos,14326.60
927,594 días 20 horas 37 minutos,14276.62
928,594 días 20 horas 12 minutos,14276.20


Año 2024:
Tickets con mayor tiempo de resolución:


,estadisticas__hora_de_resolucion,resolucion_horas_decimal
5,493 días 15 horas 36 minutos,11847.60
16,471 días 23 horas 52 minutos,11327.87
9,452 días 7 horas 40 minutos,10855.67
6,434 días 22 horas 16 minutos,10438.27
80,422 días 22 horas 26 minutos,10150.43


Año 2025:
Tickets con mayor tiempo de resolución:


,estadisticas__hora_de_resolucion,resolucion_horas_decimal
8225,163 días 3 horas 15 minutos,3915.25
3673,157 días 8 horas 23 minutos,3776.38
7400,153 días 5 horas 23 minutos,3677.38
11356,125 días 20 horas 38 minutos,3020.63
8777,125 días 6 horas 17 minutos,3006.28


In [10]:
#separacion de fechas
colums_division_fecha = ['fecha_de_apertura']

for year, df in dict_df.items():
    for col in colums_division_fecha:
        df[col + '_dia'] = df[col].dt.day
        df[col + '_mes'] = df[col].dt.month
        df[col + '_año'] = df[col].dt.year
        df[col + '_hora'] = df[col].dt.hour
        df[col + '_periodo_mes'] = df[col].dt.to_period('M')

In [11]:
#flag analisis creacion de ticket en horario laboral
for year, df in dict_df.items():
    df['creado_en_horario_laboral'] = (
        (df['fecha_de_apertura'].dt.dayofweek < 5) & 
        (df['fecha_de_apertura'].dt.hour >= 8) & 
        (df['fecha_de_apertura'].dt.hour <= 18)
    )

In [12]:
#analisis de outliers

for year, df in dict_df.items():
    limite_superior = df['resolucion_horas_decimal'].quantile(0.99)
    print(f"El 99% de los tickets de {year} se resuelven en menos de {limite_superior:.2f} horas.")
    df_anomalias = df[df['resolucion_horas_decimal'] > limite_superior]
    print(f"Se encontraron {len(df_anomalias)} tickets anómalos en el año {year}.\n")

#Reemplazar si se quiere filtrar por outliers
"""
for year, df in dict_df.items():
    limite_superior = df['resolucion_horas_decimal'].quantile(0.99)
    df = df[df['resolucion_horas_decimal'] > limite_superior]
"""

El 99% de los tickets de 2021 se resuelven en menos de 13811.94 horas.
Se encontraron 1 tickets anómalos en el año 2021.

El 99% de los tickets de 2022 se resuelven en menos de 3028.29 horas.
Se encontraron 39 tickets anómalos en el año 2022.

El 99% de los tickets de 2023 se resuelven en menos de 4209.50 horas.
Se encontraron 363 tickets anómalos en el año 2023.

El 99% de los tickets de 2024 se resuelven en menos de 1796.52 horas.
Se encontraron 351 tickets anómalos en el año 2024.

El 99% de los tickets de 2025 se resuelven en menos de 694.29 horas.
Se encontraron 366 tickets anómalos en el año 2025.



"\nfor year, df in dict_df.items():\n    limite_superior = df['resolucion_horas_decimal'].quantile(0.99)\n    df = df[df['resolucion_horas_decimal'] > limite_superior]\n"

In [13]:
#Extracción de Área y Servicio con Corrección de Texto
def reparar_texto(texto):
    """Corrije errores de codificación comunes (mojibake) en el texto."""
    if not isinstance(texto, str):
        return texto
    
    reemplazos = {
        'Ã³': 'ó', 'Ã¡': 'á', 'Ã©': 'é', 'Ã\xad': 'í', 'Ã': 'í',
        'Ã±': 'ñ', 'Ãº': 'ú', 'Ã\x81': 'Á', 'Ã\x89': 'É',
        'Ã\x8d': 'Í', 'Ã\x93': 'Ó', 'Ã\x9a': 'Ú', 'Ã\x91': 'Ñ',
        'â\x80\x93': '-', 'â\x80\x9c': '"', 'â\x80\x9d': '"'
    }
    
    for mal, bien in reemplazos.items():
        if mal in texto:
            texto = texto.replace(mal, bien)
            
    return texto.strip()

def procesar_jerarquia(df):
    if 'categoria' not in df.columns:
        return df

    series_cat = df['categoria'].fillna('Sin Categoria').astype(str)

    df['area'] = series_cat.apply(
        lambda x: x.split('>')[0].strip() if '>' in x else x.strip()
    )

    def obtener_servicio(texto):
        if '>' in texto:
            return texto.split('>')[1].strip()
        else:
            return "General"

    df['servicio'] = series_cat.apply(obtener_servicio)
    
    df['area'] = df['area'].apply(reparar_texto)
    df['servicio'] = df['servicio'].apply(reparar_texto)

    return df

#procesamiento
for year, df in dict_df.items():
    dict_df[year] = procesar_jerarquia(df)
    print(f"Año {year}: Área y Servicio extraídos y corregidos.")

print("\ndatos procesados (Año 2024)")
cols_ver = ['categoria', 'area', 'servicio']
if len(dict_df[2024]) > 0:
    display(dict_df[2024][cols_ver].sample(5, random_state=42))


Año 2021: Área y Servicio extraídos y corregidos.
Año 2022: Área y Servicio extraídos y corregidos.
Año 2023: Área y Servicio extraídos y corregidos.
Año 2024: Área y Servicio extraídos y corregidos.
Año 2025: Área y Servicio extraídos y corregidos.

datos procesados (Año 2024)


,categoria,area,servicio
30227,MDA > Llamado Cortado / Liberado,MDA,Llamado Cortado / Liberado
32968,Soporte de Campo > Requerimiento,Soporte de Campo,Requerimiento
33723,DBA > Cambio Password,DBA,Cambio Password
12465,DBA > Query BD,DBA,Query BD
20042,DBA > Cambio Password,DBA,Cambio Password


In [14]:
# Cálculo de Tiempo de Vida (Real)

for year, df in dict_df.items():
    if 'fecha_de_apertura' in df.columns and 'fecha_de_cierre' in df.columns:
        df['fecha_de_apertura'] = pd.to_datetime(df['fecha_de_apertura'], dayfirst=True, errors='coerce')
        df['fecha_de_cierre'] = pd.to_datetime(df['fecha_de_cierre'], dayfirst=True, errors='coerce')

        try:
            if pd.api.types.is_datetime64tz_dtype(df['fecha_de_apertura']):
                df['fecha_de_apertura'] = df['fecha_de_apertura'].dt.tz_convert(None)
        except Exception:
            pass
        try:
            if pd.api.types.is_datetime64tz_dtype(df['fecha_de_cierre']):
                df['fecha_de_cierre'] = df['fecha_de_cierre'].dt.tz_convert(None)
        except Exception:
            pass

        td = df['fecha_de_cierre'] - df['fecha_de_apertura']
        df['tiempo_vida_horas'] = td.dt.total_seconds() / 3600
        df['tiempo_vida_horas'] = df['tiempo_vida_horas'].round(2)

        errores = df['tiempo_vida_horas'].lt(0).sum()
        if errores > 0:
            print(f"Año {year}: ⚠️ ¡Alerta! Se encontraron {errores} tickets con fecha de cierre anterior a la apertura.")
        else:
            print(f"Año {year}: Cálculo de tiempo de vida correcto.")
    else:
        print(f"Año {year}: Faltan columnas de fecha (no se puede calcular tiempo de vida).")

# procesamiento
print("\nEjemplo 2024 (Primeras 5 filas):")
cols = ['fecha_de_apertura','fecha_de_cierre','tiempo_vida_horas', "resolucion_horas_decimal", "estadisticas__hora_de_resolucion"]
cols_presentes = [c for c in cols if c in dict_df[2024].columns]
display(dict_df[2024][cols_presentes].head())

Año 2021: Cálculo de tiempo de vida correcto.
Año 2022: Cálculo de tiempo de vida correcto.
Año 2023: Cálculo de tiempo de vida correcto.
Año 2024: Cálculo de tiempo de vida correcto.
Año 2025: Cálculo de tiempo de vida correcto.

Ejemplo 2024 (Primeras 5 filas):


/tmp/ipykernel_236126/1242335424.py:9: Pandas4Warning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if pd.api.types.is_datetime64tz_dtype(df['fecha_de_apertura']):
/tmp/ipykernel_236126/1242335424.py:14: Pandas4Warning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if pd.api.types.is_datetime64tz_dtype(df['fecha_de_cierre']):
/tmp/ipykernel_236126/1242335424.py:9: Pandas4Warning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if pd.api.types.is_datetime64tz_dtype(df['fecha_de_apertura']):
/tmp/ipykernel_236126/1242335424.py:14: Pandas4Warning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if pd.api.types.is_datetime64tz_dtype(df['fecha_de_c

,fecha_de_apertura,fecha_de_cierre,tiempo_vida_horas,resolucion_horas_decimal,estadisticas__hora_de_resolucion
0,2024-12-17 17:45:00,NaT,NaN,0.00,0 segundos
1,2024-12-10 09:04:00,NaT,NaN,0.00,0 segundos
2,2024-12-23 17:32:00,NaT,NaN,0.00,0 segundos
3,2024-05-24 14:49:00,2024-05-27 08:29:00,65.67,0.17,10 minutos
4,2024-04-17 18:00:00,2024-05-02 22:20:00,364.33,311.28,12 días 23 horas 17 minutos


In [15]:
#Eliminación de Incoherencias Temporales

for year, df in dict_df.items():
    if 'tiempo_vida_horas' in df.columns:
        n_incoherentes = df[df['tiempo_vida_horas'] < 0].shape[0]
        
        if n_incoherentes > 0:
            print(f"Año {year}: Eliminando {n_incoherentes} tickets con tiempo negativo (cierre < apertura).")
            condicion_logica = (df['tiempo_vida_horas'] >= 0) | (df['tiempo_vida_horas'].isna())
            dict_df[year] = df[condicion_logica].copy()
            print(f"  -> Registros restantes: {dict_df[year].shape[0]}")
        else:
            print(f"Año {year}: Sin incoherencias temporales.")

Año 2021: Sin incoherencias temporales.
Año 2022: Sin incoherencias temporales.
Año 2023: Sin incoherencias temporales.
Año 2024: Sin incoherencias temporales.
Año 2025: Sin incoherencias temporales.


In [16]:
# Transformación de Prioridad a Escala Numérica
mapa_prioridad = {
    'muy baja': 1,
    'baja': 2,
    'media': 3,
    'alta': 4,
    'muy alta': 5,
    'mayor': 6
}

for year, df in dict_df.items():
    if 'prioridad' in df.columns:
        # Normalizamos el texto: minúsculas y sin espacios extra para asegurar coincidencia
        # map(mapa_prioridad) transformará el texto en el número correspondiente
        df['prioridad_num'] = df['prioridad'].astype(str).str.lower().str.strip().map(mapa_prioridad)
        
        # Gestión de valores no encontrados (NaN):
        # Si hay una prioridad que no está en el mapa, quedará como NaN.
        # Podemos rellenarlos con la moda (el valor más común) o un valor medio (2).
        # Aquí usaremos 2 (Media) como valor por defecto conservador.
        nulos_antes = df['prioridad_num'].isna().sum()
        df['prioridad_num'] = df['prioridad_num'].fillna(2)
        
        print(f"Año {year}: Prioridades convertidas. (Se imputaron {nulos_antes} valores desconocidos)")

# Verificación rápida
print("\nEjemplo 2024 (Primeras 5 filas):")
display(dict_df[2024][['prioridad', 'prioridad_num']].head())

Año 2021: Prioridades convertidas. (Se imputaron 0 valores desconocidos)
Año 2022: Prioridades convertidas. (Se imputaron 0 valores desconocidos)
Año 2023: Prioridades convertidas. (Se imputaron 0 valores desconocidos)
Año 2024: Prioridades convertidas. (Se imputaron 0 valores desconocidos)
Año 2025: Prioridades convertidas. (Se imputaron 0 valores desconocidos)

Ejemplo 2024 (Primeras 5 filas):


,prioridad,prioridad_num
0,Media,3
1,Media,3
2,Media,3
3,Muy alta,5
4,Media,3


In [17]:
#eliminar columnas
cols_a_eliminar = ['unnamed_16', "estadisticas__hora_de_resolucion", "resolucion_horas_decimal"]

for year, df in dict_df.items():
    df.drop(cols_a_eliminar, axis=1, inplace=True)

In [18]:
#comprobacion de arreglos generados
for year, df in dict_df.items():
    print(f"año {year}:")
    display(df.info())
    print("----"*10)

año 2021:
<class 'pandas.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 26 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   id                                    2 non-null      int64         
 1   titulo                                2 non-null      str           
 2   estado                                2 non-null      str           
 3   ultima_modificacion                   2 non-null      datetime64[us]
 4   fecha_de_apertura                     2 non-null      datetime64[us]
 5   prioridad                             2 non-null      str           
 6   solicitante__solicitante              2 non-null      str           
 7   asignado_a__grupo_de_tecnicos         2 non-null      str           
 8   asignado_a__tecnico                   2 non-null      str           
 9   categoria                             2 non-null      str           
 10  origen_

None

----------------------------------------
año 2022:
<class 'pandas.DataFrame'>
RangeIndex: 3827 entries, 0 to 3826
Data columns (total 26 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   id                                    3827 non-null   int64         
 1   titulo                                3827 non-null   str           
 2   estado                                3827 non-null   str           
 3   ultima_modificacion                   3827 non-null   datetime64[us]
 4   fecha_de_apertura                     3827 non-null   datetime64[us]
 5   prioridad                             3827 non-null   str           
 6   solicitante__solicitante              3772 non-null   str           
 7   asignado_a__grupo_de_tecnicos         3561 non-null   str           
 8   asignado_a__tecnico                   2580 non-null   str           
 9   categoria                         

None

----------------------------------------
año 2023:
<class 'pandas.DataFrame'>
RangeIndex: 36259 entries, 0 to 36258
Data columns (total 26 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   id                                    36259 non-null  int64         
 1   titulo                                36259 non-null  str           
 2   estado                                36259 non-null  str           
 3   ultima_modificacion                   36259 non-null  datetime64[us]
 4   fecha_de_apertura                     36259 non-null  datetime64[us]
 5   prioridad                             36259 non-null  str           
 6   solicitante__solicitante              35899 non-null  str           
 7   asignado_a__grupo_de_tecnicos         33944 non-null  str           
 8   asignado_a__tecnico                   30251 non-null  str           
 9   categoria                       

...

----------------------------------------
año 2024:
<class 'pandas.DataFrame'>
RangeIndex: 35004 entries, 0 to 35003
Data columns (total 26 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   id                                    35004 non-null  int64         
 1   titulo                                35004 non-null  str           
 2   estado                                35004 non-null  str           
 3   ultima_modificacion                   35004 non-null  datetime64[us]
 4   fecha_de_apertura                     35004 non-null  datetime64[us]
 5   prioridad                             35004 non-null  str           
 6   solicitante__solicitante              34858 non-null  str           
 7   asignado_a__grupo_de_tecnicos         33346 non-null  str           
 8   asignado_a__tecnico                   33356 non-null  str           
 9   categoria                       

None

----------------------------------------
año 2025:
<class 'pandas.DataFrame'>
RangeIndex: 36527 entries, 0 to 36526
Data columns (total 26 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   id                                    36527 non-null  int64         
 1   titulo                                36527 non-null  str           
 2   estado                                36527 non-null  str           
 3   ultima_modificacion                   36527 non-null  datetime64[us]
 4   fecha_de_apertura                     36527 non-null  datetime64[us]
 5   prioridad                             36527 non-null  str           
 6   solicitante__solicitante              36369 non-null  str           
 7   asignado_a__grupo_de_tecnicos         35292 non-null  str           
 8   asignado_a__tecnico                   35160 non-null  str           
 9   categoria                       

None

----------------------------------------


In [19]:
#guardar datos procesados
%store -r folder_p

for year, df in dict_df.items():
    name = f"df{year}.csv"
    df.to_csv(f"{folder_p}/{name}")